#### Khai báo thư viện

In [2]:
!pip install vnstock

  Using cached pydantic-2.11.4-py3-none-any.whl.metadata (66 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached typing_inspection-0.4.0-py3-none-any.whl.metadata (2.6 kB)
Using cached pydantic-2.11.4-py3-none-any.whl (443 kB)
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.0 MB ? eta -:--:--
   ---------------- ----------------------- 0.8/2.0 MB 799.2 kB/s eta 0:00:02
   ------------------------------------- -- 1.8/2.0 MB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 1.5 MB/s eta 0:00:00
Using cached tenacity-9.1.2-py3-none-any.whl (28 kB)
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Using cached typing_inspection-0.4.0-py3-none-any.w


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
from vnstock import Vnstock
import pandas as pd
import json

stock = Vnstock(source="VCI").stock()
df_list_stock = pd.DataFrame(stock.listing.symbols_by_industries())
# df_list_post = pd.read_csv('cleaned_posts.csv')

2025-05-20 09:05:45 - vnstock.common.vnstock - INFO - Mã chứng khoán không được chỉ định, chương trình mặc định sử dụng VN30F1M
2025-05-20 09:05:45 - vnstock.common.data.data_explorer - INFO - Không phải là mã chứng khoán, thông tin công ty và tài chính không khả dụng.


#### Hàm lấy dữ liệu cụ thể
Những hàm sau lấy các thông tin về công ty đại diện cho các mã chứng khoán, lấy từ các nguồn nhờ thư viện `vnstock3`:
- `get_info_industry`: Lấy thông tin về tên công ty, ngành của công ty
- `get_info_overview`: Lấy thông tin về số lượng cổ đông, số lượng nhân viên, ...
- `get_info_finance`: Lấy thông tin về tài chính của công ty, các chỉ số...

In [3]:
def get_info_industry(symbol: str):
    # Lấy hàng chứa thông tin của mã cổ phiếu cần tìm
    df_row = df_list_stock[df_list_stock['symbol'] == symbol]
    df_row = df_row[['symbol', 'organ_name', 'icb_code2', 'icb_name2']]
    df_row.columns = ['symbol', 'name', 'indus_code', 'indus_name']

    return df_row

get_info_industry('VCI')

,symbol,name,indus_code,indus_name
1411,VCI,Công ty Cổ phần Chứng khoán Vietcap,8700,Dịch vụ tài chính


In [4]:
def get_info_overview(symbol: str):
    # Lấy thông tin tổng quan của mã cổ phiếu cần tìm, từ TCBS
    df_info_ovw = pd.DataFrame(Vnstock(show_log=False).stock(symbol=symbol, source='TCBS').company.overview())
    df_info_ovw['symbol'] = symbol
    df_info_ovw = df_info_ovw[['symbol', 'exchange', 'no_shareholders', 'no_employees', 'stock_rating']]

    # Fill NaN thành -1
    df_info_ovw = df_info_ovw.fillna(-1).infer_objects(copy=False)

    # Đổi tên cột
    df_info_ovw.columns = ['symbol', 'exc', 'm_stholder', 'm_employee', 'r_tcbsrating']
    
    return df_info_ovw

get_info_overview('VCI')


,symbol,exc,m_stholder,m_employee,r_tcbsrating
0,VCI,HOSE,15669,389,2.4


In [5]:
def get_info_finance(symbol: str):
    # Thử lấy thông tin tài chính từ VCI
    df_row = pd.DataFrame()
    try:
        df_row = pd.DataFrame(Vnstock(show_log=False).stock(symbol=symbol, source='VCI').finance.ratio(period='year', lang='vi'))
    except:
        pass
    
    df_info_finance = pd.DataFrame()

    # Lấy các chỉ tiêu cơ bản trong năm gần nhất (2023)
    df_row = df_row.head(1)

    LIST_KEYS = [(('Meta', 'CP')                                      , 'symbol' ), 
                 (('Meta', 'Năm')                                     , 'year'   ),
                 (('Chỉ tiêu định giá', 'Vốn hóa (Tỷ đồng)')          , 'm_cap'  ),     # m_cap
                 (('Chỉ tiêu định giá', 'Số CP lưu hành (Triệu CP)')  , 'm_share'),     # m_share
                 (('Chỉ tiêu định giá', 'P/B')                        , 'r_pb'   ),     # r_pb      = giá thị trường / BVPS
                 (('Chỉ tiêu định giá', 'P/E')                        , 'r_pe'   ),     # r_pe      = giá thị trường / lợi nhuận sau thuế
                 (('Chỉ tiêu định giá', 'EPS (VND)')                  , 'r_eps'  ),     # r_eps     = lợi nhuận sau thuế / số CP lưu hành
                 (('Chỉ tiêu định giá', 'BVPS (VND)')                 , 'r_bvps' ),     # r_bvps    = vốn chủ sở hữu / số CP lưu hành
                 (('Chỉ tiêu khả năng sinh lợi', 'ROE (%)')           , 'p_roe'  ),     # p_roe     = lợi nhuận sau thuế / vốn chủ sở hữu
                 (('Chỉ tiêu khả năng sinh lợi', 'Tỷ suất cổ tức (%)'), 'p_div'  ),     # p_div     = Tỷ suất cổ tức = cổ tức / giá thị trường
                ]

    # Kiểm tra xem các chỉ tiêu có trong df_info_finance không
    for key, col_name in LIST_KEYS:
        if key not in df_row.columns:
            df_row[key] = 0

        # Đổi tên cột thanh col_name
        df_info_finance[col_name] = df_row[key]

    return df_info_finance

get_info_finance('VCI')

,symbol,year,m_cap,m_share,r_pb,r_pe,r_eps,r_bvps,p_roe,p_div
0,VCI,2024,26354250916000,718099480,2.035973,28.938706,1268.197706,18025.783068,0.089655,0.017711


In [6]:
# Lấy toàn bộ mã cổ phiếu trong list_stock
list_symbol = df_list_stock['symbol'].tolist()
df_info = pd.DataFrame()

for i, symbol in enumerate(list_symbol):
    # Lấy thông tin của mã cổ phiếu ở 3 hàm
    df_info_industry = get_info_industry(symbol)
    df_info_ovw = get_info_overview(symbol)
    df_info_finance = get_info_finance(symbol)

    # Gộp
    df_info_temp = pd.merge(df_info_industry, df_info_ovw, on='symbol', how='inner')
    df_info_temp = pd.merge(df_info_temp, df_info_finance, on='symbol', how='inner')

    df_info = pd.concat([df_info, df_info_temp], ignore_index=True)

    # Lưu vào file CSV
    df_info_temp.to_csv('list_stock.csv', index=False, encoding='utf-8', header=(False if i > 0 else True), mode='a')
    print(f'Processed {symbol} ({i+1}/{len(list_symbol)})!', end='\r')

print(f'\nProcessed {len(df_info)} stocks, end of data')

df_info = df_info.reset_index(drop=True)
df_info.tail(20)

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


...

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


Processed WSB (1594/1594)!
Processed 1569 stocks, end of data


,symbol,name,indus_code,indus_name,exc,m_stholder,m_employee,r_tcbsrating,year,m_cap,m_share,r_pb,r_pe,r_eps,r_bvps,p_roe,p_div
1549,DFC,Công ty Cổ phần Xích líp Đông Anh,1700,Tài nguyên Cơ bản,UPCOM,795,1296,-1.0,2024,3.409740e+11,1.140000e+07,1.373422,8.869664,3512.519291,21777.719825,0.166018,0.000000
1550,VTC,Công ty Cổ phần Viễn Thông VTC,9500,Công nghệ Thông tin,HNX,1465,127,-1.0,2024,3.763798e+10,4.534696e+06,0.551620,11.871680,777.631175,15046.600043,0.051428,0.000000
1551,XMC,Công ty cổ phần Đầu tư và Xây dựng Xuân Mai,2300,Xây dựng và Vật liệu,UPCOM,1300,560,2.2,2024,4.240784e+11,7.140569e+07,0.488669,54.207756,109.559968,12153.426510,0.008789,0.000000
1552,XMD,Công ty Cổ phần Xuân Mai - Đạo Tú,2300,Xây dựng và Vật liệu,UPCOM,488,173,-1.0,2024,2.680000e+10,4.000000e+06,0.630796,8.331015,804.223725,10621.506363,0.078696,0.000000
1553,VTE,Công ty Cổ phần VINACAP Kim Long,9500,Công nghệ Thông tin,UPCOM,474,108,-1.0,2024,6.240000e+10,1.560000e+07,0.368246,154.187816,25.942387,10862.302028,0.002383,0.000000
1554,VGG,Tổng Công Ty Cổ phần May Việt Tiến,3700,Hàng cá nhân & Gia dụng,UPCOM,1425,4275,2.9,2024,1.803073e+12,4.410000e+07,0.875436,5.746827,7958.588517,46703.567425,0.177558,0.073375
1555,CQT,Công ty Cổ phần Xi măng Quán Triều VVMI,2300,Xây dựng và Vật liệu,UPCOM,585,350,-1.0,2024,2.125000e+11,2.500000e+07,0.816635,99.710922,158.778763,10408.566194,0.014588,0.000000
1556,VGI,Tổng Công ty Cổ phần Đầu tư Quốc tế Viettel,6500,Viễn thông,UPCOM,9648,5045,2.5,2024,2.160162e+14,3.043811e+09,6.244232,38.394871,1848.397934,11365.529748,0.174128,0.000000
1557,VTH,Công ty Cổ phần Dây cáp điện Việt Thái,2700,Hàng & Dịch vụ Công nghiệp,HNX,0,127,-1.0,2024,6.635991e+10,7.899989e+06,0.679192,7.579562,1166.572006,12367.628506,0.098993,0.000000
1558,XPH,Công ty Cổ phần Xà phòng Hà Nội,3700,Hàng cá nhân & Gia dụng,UPCOM,0,66,-1.0,2024,1.226807e+11,1.297248e+07,0.848763,-17.116449,-552.509459,11142.090954,-0.048388,0.000000


In [7]:
# Danh sách ngành
df_list_indus = df_list_stock[['icb_code2', 'icb_name2']].drop_duplicates().reset_index(drop=True)
df_list_indus

,icb_code2,icb_name2
0,2300,Xây dựng và Vật liệu
1,5500,Truyền thông
2,7500,"Điện, nước & xăng dầu khí đốt"
3,2700,Hàng & Dịch vụ Công nghiệp
4,3300,Ô tô và phụ tùng
5,4500,Y tế
6,1300,Hóa chất
7,3500,Thực phẩm và đồ uống
8,5700,Du lịch và Giải trí
9,8700,Dịch vụ tài chính


#### Hàm lấy giá lịch sử

Hàm này sẽ lấy giá lịch sử của các mã được đề cập đến trong những bài viết trong file `cleaned_posts.csv`.
- Khung thời gian lấy sẽ là 30 phút, trong trường hợp không có đủ dữ liệu thì dùng khung 1 ngày.
- Bắt đầu lấy dữ liệu từ ngày `01/12/2024` đến `11/04/2025`.

In [8]:
def process_symbol(cmponent, symb, interval='1D'):
    try:
        df = pd.DataFrame(cmponent.quote.history(symbol=symb, start='2024-12-01', end='2025-04-11', interval = interval))
        if len(df) == 0:
            raise Exception('No data')
        
        # Xóa các dòng  không cần thiết
        df = df.dropna()

        # Lưu vào file CSV
        df.to_csv('price/' + symb + '.csv', index=False)
        return 1
    except Exception as e:
        print(f'\nError processing {symb}: {e}')
        return 0

In [9]:
# Lấy danh sách mã cổ phiếu được đề cập
# Công nghệ		    FPT
# Chứng khoán		    SSI
# Ngân hàng		    VCB
# Bất động sản		VHM
# Vật liệu cơ bản		HPG
# Dịch vụ Hạ tầng		GAS
# Tiêu dùng cơ bản	MSN
# Bán lẻ			    MWG
# Chế biến		    GVR
# Công nghiệp		    VCG


list_taggedSymbols = ["FPT", "SSI", "VCB", "VHM", "HPG", "GAS", "MSN", "MWG", "GVR", "VCG"]
# set_tagg

# for post in df_list_post['taggedSymbols']:
#     symb_array = json.loads(post)
#     for symb in symb_array:
#         set_taggedSymbols.add(symb['symb'])

# list_taggedSymbols = list(set_taggedSymbols)
list_taggedSymbols.sort()

print(list_taggedSymbols)

['FPT', 'GAS', 'GVR', 'HPG', 'MSN', 'MWG', 'SSI', 'VCB', 'VCG', 'VHM']


In [11]:
# Kiểm tra xem thư mục price có tồn tại không, nếu không thì tạo mới
import os
if not os.path.exists('price'):
    os.makedirs('price')

count_processed = 0

for i, symb in enumerate(list_taggedSymbols):

    # # Phân loại mã cổ phiếu
    # if symb[0] == '$':  # Tiền điện tử (crypto)
    #     crypto = Vnstock().crypto(source='MSN')
    #     count_processed += process_symbol(crypto, symb[1:])
    # elif symb[0] == '^':  # Chỉ số (index)
    #     index = Vnstock().world_index(source='MSN')
    #     count_processed += process_symbol(index, symb[1:])
    # elif '-' in symb:     # Tỷ giá (forex)
    #     forex = Vnstock().fx(source='MSN')
    #     count_processed += process_symbol(forex, symb.replace('-', ''))
    # elif len(symb) > 3 and symb[0] == 'C': # Chứng quyền (warrant), bỏ qua
    #     continue
    # elif len(symb) > 3 and (symb[-2::] == '24' or symb[-2::] == '25'): # Hợp đồng tương lai (future), bỏ qua
    #     continue
    # elif '=' in symb or '.' in symb:     # Một số mã cổ phiếu khác, bỏ qua
    #     continue
    # else:
    count_processed += process_symbol(stock, symb) # Mã cổ phiếu thông thường, lấy dữ liệu 30 phút
    print(f'Processed {symb} ({i+1}/{len(list_taggedSymbols)}), saved to price/1d/{symb}.csv', end='\r')

print(f'\nProcessed {count_processed} stocks, end of data')


Processed VHM (10/10), saved to price/1d/VHM.csv
Processed 10 stocks, end of data
